In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

# ! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG


In [ ]:
# import os

# if not os.getenv("IS_TESTING"):
#     # Automatically restart kernel after installs
#     import IPython

#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"


In [1]:
import sys
from pathlib import Path
from datetime import datetime 
sys.path.append(str(Path(".").absolute().parent))
sys.path.append(str(Path(".").absolute().parent) + "/utils")
sys.path.append(str(Path(".").absolute().parent.parent))
sys.path.append(str(Path(".").absolute().parent.parent.parent))
import pipeline_utils
PARAMS = pipeline_utils.yaml_import('settings.yml')

ENV = PARAMS['env_flag']

PROJECT_ID = PARAMS['envs'][ENV]['PROJECT_ID']
REGION = PARAMS['envs'][ENV]['REGION']
BASE_IMAGE = PARAMS['envs'][ENV]['BASE_IMAGE']
MLFLOW_IMAGE = PARAMS['envs'][ENV]['MLFLOW_IMAGE']

PIPELINE_ROOT = PARAMS['envs'][ENV]['PIPELINE_ROOT']
PIPELINE_NAME = PARAMS['envs'][ENV]['PIPELINE_NAME']
PIPELINE_JSON = PARAMS['envs'][ENV]['PIPELINE_JSON']
# TMP_PIPELINE_JSON = os.path.join("/tmp", PIPELINE_JSON)


TRAINING_TABLE_NAME = PARAMS['envs'][ENV]['TRAINING_TABLE_NAME']
TRAINING_DATA_BQ_QUERY = f'select * from {TRAINING_TABLE_NAME}'

MLFLOW_EXP_NAME = PARAMS['envs'][ENV]['MLFLOW_EXP_NAME']
MODEL_REGISTRY_NAME = PARAMS['envs'][ENV]['MODEL_REGISTRY_NAME']

SERVICE_ACCOUNT = PARAMS['envs'][ENV]['SERVICE_ACCOUNT']

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
 
# Matches on non-word, non-regular-punctuation characters.
# MATCHER = r"""[^a-zA-Z0-9'"!@#$%\^&*()\[\]{}:;<>?,.-=_+ ]+""" 

CLUB_THRESH_PATH = PARAMS['envs'][ENV]['CLUB_THRESH_PATH']
LATEST_NOSALES_MODEL_PATH = PARAMS['envs'][ENV]['LATEST_NOSALES_MODEL_PATH']
LATEST_PIPELINE_PATH = PARAMS['envs'][ENV]['LATEST_PIPELINE_PATH']
# RUN_PIPELINE = PARAMS['envs'][ENV]['RUN_PIPELINE']
# print(f"ENV: {ENV}, \nPROJECT_ID: {PROJECT_ID}, \nBASE_IMAGE: {BASE_IMAGE}, \nMLFLOW_IMAGE: {MLFLOW_IMAGE}, \nPIPELINE_NAME: {PIPELINE_NAME}, \nPIPELINE_JSON: {PIPELINE_JSON}")


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "wmt-mlp-p-oyi-ds-or-oyi-dsns":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID


In [ ]:
! gcloud config get-value project

In [ ]:
REGION

In [ ]:
SERVICE_ACCOUNT

In [ ]:

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "svc-oyi-ds-or-oyi-dsns-admin@wmt-mlp-p-oyi-ds-or-oyi-dsns.iam.gserviceaccount.com"
):

    # Get your GCP project id from gcloud

    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    print("Service Account:", SERVICE_ACCOUNT)

In [ ]:
#Create the json key file of the service account.
!gcloud iam service-accounts keys create sa.json --iam-account={SERVICE_ACCOUNT}

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/home/jupyter/test/sa.json'

In [ ]:
#Authenticating service account for the upcoming operations.
!gcloud auth activate-service-account --key-file=sa.json
!gcloud config set account {SERVICE_ACCOUNT}

In [53]:
# shell_output = !gcloud auth list 2>/dev/null
# shell_output[3].replace("*", "").strip()

In [2]:
BUCKET_URI = PIPELINE_ROOT
print(BUCKET_URI)

gs://oyi-ds-vertex-pipeline-bucket-nonprod


In [3]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == PIPELINE_ROOT:
    BUCKET_URI = "gs://" + PROJECT_ID + "-aip-" + TIMESTAMP
    print(BUCKET_URI)

gs://wmt-mlp-p-oyi-ds-or-oyi-dsns-aip-20221017234510


In [4]:
! gsutil mb -l $REGION $BUCKET_URI


Creating gs://wmt-mlp-p-oyi-ds-or-oyi-dsns-aip-20221017234510/...


In [5]:
! gsutil ls -al $BUCKET_URI


In [6]:
! gsutil ls -al 

gs://artifacts.wmt-mlp-p-oyi-ds-or-oyi-dsns.appspot.com/
gs://oyi-ds-club-score-cutoff-pipeline-bucket/
gs://oyi-ds-club-score-cutoff-pipeline-bucket-nonprod/
gs://oyi-ds-vertex-experiment-bucket/
gs://oyi-ds-vertex-output-bucket/
gs://oyi-ds-vertex-pipeline-bucket/
gs://oyi-ds-vertex-pipeline-bucket-nonprod/
gs://oyi-ds-vertex-pipeline-bucket-output/
gs://oyi-ds-vertex-test-input-bucket/
gs://wmt-mlp-p-oyi-ds-or-oyi-dsns-aip-20221017223909/
gs://wmt-mlp-p-oyi-ds-or-oyi-dsns-aip-20221017234510/
gs://wmt-mlp-p-oyi-ds-or-oyi-dsns-bq-temp/
gs://wmt-mlp-p-oyi-ds-or-oyi-dsns-export-bucket/


In [7]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

In [8]:
import google.cloud.aiplatform as aip


In [9]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
print(API_ENDPOINT)

us-central1-aiplatform.googleapis.com


In [10]:
_PIPELINE_ROOT = "{}/pipeline_root/intro".format(BUCKET_URI)
print(_PIPELINE_ROOT)

gs://wmt-mlp-p-oyi-ds-or-oyi-dsns-aip-20221017234510/pipeline_root/intro


In [11]:
from typing import NamedTuple

from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component

In [12]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)


In [13]:
%run pipeline.py

ENV: dev 
PROJECT_ID: wmt-mlp-p-oyi-ds-or-oyi-dsns 
BASE_IMAGE: gcr.io/wmt-mlp-p-oyi-ds-or-oyi-dsns/oyi-vertex-pipeline-dev:latest 
MLFLOW_IMAGE: gcr.io/wmt-mlp-p-oyi-ds-or-oyi-dsns/mlflow-image-dev:latest 
PIPELINE_NAME: oyi-nosales-model-pipeline-dev 
PIPELINE_JSON: oyi-nosales-model-pipeline-dev.json

TMP_PIPELINE_JSON: /tmp/oyi-nosales-model-pipeline-dev.json 
LATEST_PIPELINE_PATH: gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev 
LATEST_PIPELINE_PATH_JSON: gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev.json


usage: pipeline.py [-h] --MODE MODE --STAGE1_FLAG STAGE1_FLAG --ENSEMBLE_FLAG
                   ENSEMBLE_FLAG --RF_CLF_MODEL_PATH RF_CLF_MODEL_PATH
                   --LOGISTIC_CLF_MODEL_PATH LOGISTIC_CLF_MODEL_PATH
                   --STAGE1_NN_MODEL_PATH STAGE1_NN_MODEL_PATH
                   --GNB_MODEL_PATH GNB_MODEL_PATH
                   --STG1_FEATURE_SELECTOR_MODEL_PATH
                   STG1_FEATURE_SELECTOR_MODEL_PATH --NOSALES_MODEL_PATH
                   NOSALES_MODEL_PATH
pipeline.py: error: the following arguments are required: --MODE, --STAGE1_FLAG, --ENSEMBLE_FLAG, --RF_CLF_MODEL_PATH, --LOGISTIC_CLF_MODEL_PATH, --STAGE1_NN_MODEL_PATH, --GNB_MODEL_PATH, --STG1_FEATURE_SELECTOR_MODEL_PATH, --NOSALES_MODEL_PATH


In [15]:
os.path.join("/tmp", "intro_pipeline.json")

'/tmp/intro_pipeline.json'

In [47]:
DISPLAY_NAME, DISPLAY_NAME_JSON, LATEST_PIPELINE_PATH_JSON, TMP_PIPELINE_JSON, _PIPELINE_ROOT

('intro_20221017234643',
 'gs://oyi-ds-vertex-pipeline-bucket-nonprod/intro_20221017234643-dev.json',
 'gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_nosales_model_output_dev.json',
 '/tmp/oyi-nosales-model-pipeline-dev.json',
 'gs://wmt-mlp-p-oyi-ds-or-oyi-dsns-aip-20221017234510/pipeline_root/intro')

In [35]:
DISPLAY_NAME = "intro_" + TIMESTAMP
DISPLAY_NAME_JSON = f"gs://oyi-ds-vertex-pipeline-bucket-nonprod/{DISPLAY_NAME}-{ENV}.json"
TMP_TEMPLATE_PATH = os.path.join("/tmp", f"intro_pipeline-{ENV}.json")

pipeline_utils.store_pipeline(
    storage_path=DISPLAY_NAME_JSON, #gs://oyi-ds-vertex-pipeline-bucket-nonprod/latest_training_pipeline_dev.json
    filename=TMP_PIPELINE_JSON
)

pipeline_job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path=TMP_PIPELINE_JSON, # Pipeline json file name
    pipeline_root=_PIPELINE_ROOT
)

# pipeline_job.run(service_account=SERVICE_ACCOUNT)

# pipeline_job = aiplatform.PipelineJob(
#     display_name=f"{PIPELINE_NAME}-{TIMESTAMP}",
#     template_path=TMP_PIPELINE_JSON,
#     pipeline_root=PIPELINE_ROOT,
#     parameter_values={},
#     enable_caching=False,
# )

# pipeline_utils.store_pipeline(
#     storage_path=LATEST_PIPELINE_PATH_JSON, 
#     filename=TMP_PIPELINE_JSON
# )


/tmp/oyi-nosales-model-pipeline-dev.json
contents /tmp/oyi-nosales-model-pipeline-dev.json uploaded to gs://oyi-ds-vertex-pipeline-bucket-nonprod/intro_20221017234643-dev.json.


In [36]:
%%writefile utils.py

import json
from typing import Any, Dict

from google.cloud import storage
from google.cloud import scheduler_v1

# Load data from a JSON document
def load_json(path: str) -> Dict[str, Any]:
    """Loads data from a JSON document.

    Args:
      path: The path of the JSON document. It can be a local path or a GS URI.

    Returns:
      A deserialized Dict object representing the JSON document.
    """

    if path.startswith('gs://'):
        return _load_json_from_gs_uri(path)
    else:
        return _load_json_from_local_file(path)

# Load data from a JSON document referenced by a GS URI
def _load_json_from_gs_uri(uri: str) -> Dict[str, Any]:
    """Loads data from a JSON document referenced by a GS URI.

    Args:
      uri: The GCS URI of the JSON document.

    Returns:
      A deserialized Dict object representing the JSON document.

    Raises:
      google.cloud.exceptions.NotFound: If the blob is not found.
      json.decoder.JSONDecodeError: On JSON parsing problems.
      ValueError: If uri is not a valid gs URI.
    """
    storage_client = storage.Client()
    blob = storage.Blob.from_string(uri, storage_client)
    return json.loads(blob.download_as_bytes())

# Load data from a JSON local file
def _load_json_from_local_file(file_path: str) -> Dict[str, Any]:
    """Loads data from a JSON local file.

    Args:
      file_path: The local file path of the JSON document.

    Returns:
      A deserialized Dict object representing the JSON document.

    Raises:
      json.decoder.JSONDecodeError: On JSON parsing problems.
    """
    with open(file_path) as f:
        return json.load(f)

# Get the scheduler job details
def get_job(job_name):
    # Create a client
    client = scheduler_v1.CloudSchedulerClient()

    # Initialize request argument(s)
    request = scheduler_v1.GetJobRequest(
        name=job_name,
    )

    # Make the request
    response = client.get_job(request=request)

    # Handle the response
    #print(response)

# Delete scheduler job by job-name
def delete_job(job_name):
    # Create a client
    client = scheduler_v1.CloudSchedulerClient()

    # Initialize request argument(s)
    request = scheduler_v1.DeleteJobRequest(
        name=job_name,
    )

    # Make the request
    client.delete_job(request=request)
    
    return True

Writing utils.py


In [37]:
%%writefile schedule.py

import base64
import hashlib
import json
import logging
import pathlib
import re
import tempfile
from typing import Any, Mapping, Optional
import zipfile

import googleapiclient
from googleapiclient import discovery
import requests
from google.cloud import resourcemanager_v3

from kfp.v2.google.client import client_utils
from kfp.v2.google.client import runtime_config_builder

_PROXY_FUNCTION_NAME = 'rahul_templated_http_request-v1'
# _PROXY_FUNCTION_FILENAME = '_cloud_function_templated_http_request.py'

_CAIPP_ENDPOINT_WITHOUT_REGION = 'aiplatform.googleapis.com'
_CAIPP_API_VERSION = 'v1beta1'

def _get_cloud_functions_api():
    functions_service = discovery.build(
        'cloudfunctions', 'v1', cache_discovery=False)
    functions_api = functions_service.projects().locations().functions()
    return functions_api

def _create_or_get_cloud_function(
    name: str,
    cloud_scheduler_service_account: str,
    cloud_function_project_id: str,
    region: str,
    runtime: str = 'python37',
):
    
    """Creates Google Cloud Function."""
    functions_api = _get_cloud_functions_api()

    project_location_path = 'projects/{}/locations/{}'.format(
        cloud_function_project_id, region)
    function_full_name = project_location_path + '/functions/' + name
    
    print("CLOUD FUNCTION FULL NAME :", function_full_name)
    
    # Returning early if the function already exists.
    try:
        function_get_response = functions_api.get(
            name=function_full_name).execute()
        
        # print("[BEFORE]: function_get_response", function_get_response)
        
        return function_get_response
    except googleapiclient.errors.HttpError as err:
        raise_error = True
        if err.resp['status'] == '404':
            # The function does not exist, which is expected.
            logging.info('["ERROR"] Cloud Function: name=%s does not exist', function_full_name)
            raise_error = False
        if raise_error:
            raise err
        return False
    
    return function_get_response

def _get_proxy_cloud_function_endpoint(
    cloud_function_project_id: str,
    cloud_scheduler_service_account: str,
    region: str = 'us-central1',
):
    function_dict = _create_or_get_cloud_function(
        name=_PROXY_FUNCTION_NAME,
        cloud_function_project_id=cloud_function_project_id,
        region=region,
        runtime='python37',
        cloud_scheduler_service_account=cloud_scheduler_service_account,
    )

    if not function_dict:
        print("Exiting the code")
        import sys
        sys.exit()
        
    else:
        endpoint_url = function_dict['httpsTrigger']['url']
        return endpoint_url

def _enable_required_apis(project_id: str,):
    """Enables necessary APIs."""
    serviceusage_service = discovery.build('serviceusage', 'v1', cache_discovery=False)
    services_api = serviceusage_service.services()

    required_services = [
        'cloudfunctions.googleapis.com',
        'cloudscheduler.googleapis.com',
        'appengine.googleapis.com',  # Required by the Cloud Scheduler.
    ]
    project_path = 'projects/' + project_id
    for service_name in required_services:
        service_path = project_path + '/services/' + service_name
        services_api.enable(name=service_path).execute()
        
def _create_scheduler_job(project_location_path: str,
                          job_body: Mapping[str, Any]) -> str:
    """Creates a scheduler job.

    Args:
      project_location_path: The project location path.
      job_body: The scheduled job dictionary object.

    Returns:
      The response from scheduler service.
    """
    # We cannot use google.cloud.scheduler_v1.CloudSchedulerClient since
    # it's not available internally.
    scheduler_service = discovery.build(
        'cloudscheduler', 'v1', cache_discovery=False)
    scheduler_jobs_api = scheduler_service.projects().locations().jobs()
    response = scheduler_jobs_api.create(
        parent=project_location_path,
        body=job_body,
    ).execute()
    return response


def _create_from_pipeline_dict(
    schedule: str,
    project_id: str,
    cloud_function_project_id: str,
    cloud_scheduler_service_account: str,
    pipeline_dict: dict = None,
    parameter_values: Optional[Mapping[str, Any]] = None,
    pipeline_root: Optional[str] = None,
    service_account: Optional[str] = None,
    app_engine_region: Optional[str] = None,
    scheduler_job_name: Optional[str] = None,
    region: str = 'us-central1',
    time_zone: str = 'US/Pacific'
) -> dict:
    """Creates schedule for compiled pipeline dictionary."""

    _enable_required_apis(project_id=project_id)

    # If appengine region is not provided, use the pipeline region.
    app_engine_region = app_engine_region or region

    proxy_function_url = _get_proxy_cloud_function_endpoint(
        cloud_function_project_id=cloud_function_project_id,
        region=region,
        cloud_scheduler_service_account=cloud_scheduler_service_account,
    )

    if parameter_values or pipeline_root:
        config_builder = runtime_config_builder.RuntimeConfigBuilder.from_job_spec_json(
            pipeline_dict)
        config_builder.update_runtime_parameters(
            parameter_values=parameter_values)
        config_builder.update_pipeline_root(pipeline_root=pipeline_root)
        updated_runtime_config = config_builder.build()
        pipeline_dict['runtimeConfig'] = updated_runtime_config

    # Creating job creation request to get the final request URL
    pipeline_jobs_api_url = f'https://{region}-{_CAIPP_ENDPOINT_WITHOUT_REGION}/{_CAIPP_API_VERSION}/projects/{project_id}/locations/{region}/pipelineJobs'
    
    # Preparing the request body for the Cloud Function processing
    pipeline_name = pipeline_dict['pipelineSpec']['pipelineInfo']['name']
    full_pipeline_name = 'projects/{}/pipelineJobs/{}'.format(project_id, pipeline_name)
    pipeline_display_name = pipeline_dict.get('displayName')
    time_format_suffix = "-{{$.scheduledTime.strftime('%Y-%m-%d-%H-%M-%S')}}"
    if 'name' in pipeline_dict:
        pipeline_dict['name'] += time_format_suffix
    if 'displayName' in pipeline_dict:
        pipeline_dict['displayName'] += time_format_suffix

    pipeline_dict['_url'] = pipeline_jobs_api_url
    pipeline_dict['_method'] = 'POST'

    if service_account is not None:
        pipeline_dict['serviceAccount'] = service_account

    pipeline_text = json.dumps(pipeline_dict)
    pipeline_data = pipeline_text.encode('utf-8')

#================ CUSTOM ==============================
    # pipeline_dict = {"pipeline_spec_uri":"gs://pipeline-schedule/intro_pipeline.json"}
    # pipeline_text = json.dumps(pipeline_dict)
    # pipeline_data = pipeline_text.encode('utf-8')

# ======================================================

    project_location_path = 'projects/{}/locations/{}'.format(
        project_id, app_engine_region)
    scheduled_job_full_name = '{}/jobs/{}'.format(project_location_path,
                                                  scheduler_job_name)
    service_account_email = cloud_scheduler_service_account or '{}@appspot.gserviceaccount.com'.format(
        project_id)

    scheduled_job = dict(
        name=scheduled_job_full_name,  # Optional. Only used for readable names.
        schedule=schedule,
        time_zone=time_zone,
        http_target=dict(
            http_method='POST',
            uri=proxy_function_url,
            # Warning: when using google.cloud.scheduler_v1, the type of body is
            # bytes or string. But when using the API through discovery, the body
            # needs to be base64-encoded.
            body=base64.b64encode(pipeline_data).decode('utf-8'),
            oidc_token=dict(service_account_email=service_account_email,audience = proxy_function_url),
        ),
        # TODO(avolkov): Add labels once Cloud Scheduler supports them
        # labels={
        #     'google.cloud.ai-platform.pipelines.scheduling': 'v1alpha1',
        # },
    )

    try:
        response = _create_scheduler_job(
            project_location_path=project_location_path,
            job_body=scheduled_job,
        )
        return response
    except googleapiclient.errors.HttpError as err:
        # Handling the case where the exact schedule already exists.
        if err.resp.get('status') == '409':
            raise RuntimeError(
                'The exact same schedule already exists') from err
        raise err

Writing schedule.py


In [38]:
%%writefile client.py

from typing import Any, Dict, List, Mapping, Optional
from schedule import _create_from_pipeline_dict
from utils import load_json, get_job, delete_job

class aipclient_custom:
    def __init__(self, project_id: str = None,region: str = None):
        self.project_id = project_id
        self.region = region
    
    def create_schedule_from_job_spec(self,
        job_spec_path: str = None,
        schedule: str = "* * * * *",
        time_zone: str = 'US/Pacific',
        pipeline_root: Optional[str] = None,
        parameter_values: Optional[Mapping[str, Any]] = None,
        cloud_function_project_id: str = None,
        service_account: Optional[str] = None,
        enable_caching: Optional[bool] = None,
        app_engine_region: Optional[str] = None,
        cloud_scheduler_service_account: Optional[str] = None,
        job_name: Optional[str] = None,
    ) -> dict:
        """Creates schedule for compiled pipeline file.

        This function creates scheduled job which will run the provided pipeline on
        schedule. This is implemented by creating a Google Cloud Scheduler Job.
        The job will be visible in https://console.google.com/cloudscheduler and can
        be paused/resumed and deleted.

        To make the system work, this function also creates a Google Cloud Function
        which acts as an intermediary between the Scheduler and Pipelines. A single
        function is shared between all scheduled jobs.
        The following APIs will be activated automatically:
        * cloudfunctions.googleapis.com
        * cloudscheduler.googleapis.com
        * appengine.googleapis.com

        Args:
          job_spec_path: Path of the compiled pipeline file.
          schedule: Schedule in cron format. Example: "45 * * * *"
          time_zone: Schedule time zone. Default is 'US/Pacific'
          parameter_values: Arguments for the pipeline parameters
          pipeline_root: Optionally the user can override the pipeline root
            specified during the compile time.
          service_account: The service account that the pipeline workload runs as.
          enable_caching: Whether or not to enable caching for the run.
            If not set, defaults to the compile time settings, which are True for all
            tasks by default, while users may specify different caching options for
            individual tasks.
            If set, the setting applies to all tasks in the pipeline -- overrides
            the compile time settings.
          app_engine_region: The region that cloud scheduler job is created in.
          cloud_scheduler_service_account: The service account that Cloud Scheduler job and the proxy cloud function use.
            this should have permission to call AI Platform API and the proxy function.
            If not specified, the functions uses the App Engine default service account.

        Returns:
          Created Google Cloud Scheduler Job object dictionary.
        """
        if job_spec_path is not None:
            job_spec = load_json(job_spec_path)
        
        if cloud_function_project_id is None:
            cloud_function_project_id = self.project_id
        if enable_caching is not None:
            _set_enable_caching_value(job_spec['pipelineSpec'], enable_caching)
        
        # To check if a job already exists with the same name
        self.check_job(job_name)
        
        scheduler_job_name = job_name.split("/")[-1]
        # Create new job
        return _create_from_pipeline_dict(
            pipeline_dict=job_spec,
            schedule=schedule,
            project_id=self.project_id,
            cloud_function_project_id = cloud_function_project_id, #cloud-function_project_id
            region=self.region,
            time_zone=time_zone,
            parameter_values=parameter_values,
            pipeline_root=pipeline_root,
            service_account=service_account,
            app_engine_region=app_engine_region,
            cloud_scheduler_service_account=cloud_scheduler_service_account,
            scheduler_job_name = scheduler_job_name)
    
    def check_job(self,job_name):
        if job_name is not None:
            try:
                get_job(job_name)
                print("Job already exists. Deleting..")
                delete_job(job_name)
                print("Creating a new scheduler job")

            except Exception as e:
                print("Job doesn't exist")
        
        else:
            print("Creating a new scheduler job")

Writing client.py


In [40]:
scheduler_job_name="Cloud_function_scheduler_job" # Any cloud scheduler job name

from client import aipclient_custom

api_client = aipclient_custom(PROJECT_ID,REGION)

CLOUD_FUNCTION_PROJECT_ID= PROJECT_ID #"<Enter the cloud function project id>"


In [42]:
TMP_PIPELINE_JSON

'/tmp/oyi-nosales-model-pipeline-dev.json'

In [43]:


api_client.create_schedule_from_job_spec(job_spec_path=TMP_PIPELINE_JSON, #"intro_pipeline.json",
    schedule="* * * * *",
    cloud_function_project_id = CLOUD_FUNCTION_PROJECT_ID,
    time_zone="America/Los_Angeles",  # change this as necessary
    parameter_values={"text": "Hello world!"},
    service_account=SERVICE_ACCOUNT, # to run the vertex pipeline
    cloud_scheduler_service_account = SERVICE_ACCOUNT,
    job_name = f"projects/{PROJECT_ID}/locations/{REGION}/jobs/{scheduler_job_name}"
)


Job doesn't exist
CLOUD FUNCTION FULL NAME : projects/wmt-mlp-p-oyi-ds-or-oyi-dsns/locations/us-central1/functions/rahul_templated_http_request-v1
Exiting the code


SystemExit: 

In [ ]:
!gcloud auth list

In [ ]:
!gcloud auth list 2>/dev/null

In [ ]:
shell_output

In [ ]:
shell_output[2].replace("*", "").strip()

In [ ]:
shell_output[2].strip()